In [12]:
from pysat.solvers import Glucose3
import itertools
class Coloring:
    def __init__(self):
        self.solver = Glucose3()
        self.row, self.col = 0, 0
        self.maze = []

    # đọc file
    def readFile(self, fileName):
        with open(fileName, 'r') as f:
            lines = f.readlines()
            self.row, self.col = map(int, lines[0].strip().split())
            for line in lines[1:]:
                for item in line.split():
                    self.maze.append(int(item)) if item.isdigit() else self.maze.append(item)
        f.close()

    # tô màu
    def red(self,text):
        return f"\033[1;37;41m{text}\033[0;0m"
    def green(self,text):
        return f"\033[1;37;42m{text}\033[0;0m" 
    
    # add clause
    def addClause(self, listPoint: list, value):
        for pairs in itertools.combinations(listPoint, value + 1):
            self.solver.add_clause(list([-x for x in pairs]))
        for pairs in itertools.combinations(listPoint, len(listPoint) - value + 1):
            self.solver.add_clause(list(pairs))

    # get neighbors point of position
    def getNeighbors(self, arr, row, col):
        return [arr[x][y] for x in range(row-1, row+2) for y in range(col-1, col+2)
                if 0 <= x < len(arr) and 0 <= y < len(arr[0]) and (x, y) != (row, col)] + [arr[row][col]]
    
    def solve(self):
        for number in range(len(self.maze)):
            if isinstance(self.maze[number], int):
                self.addClause(
                    self.getNeighbors( [[num for num in range((i * self.col) + 1, (i + 1) * self.col + 1)] for i in range(self.row)], number//self.col, number%self.col)
                                ,self.maze[number])
        # in kết quả
        if self.solver.solve():
            model = self.solver.get_model()
            for number in range(len(self.maze)):
                text = str(self.maze[number]) if isinstance(self.maze[number], int) else " "
                print(self.green( text + " "), end = '') if model[number] > 0 else print(self.red( text + " "), end = '')
                if (number + 1) % self.col == 0:
                    print() 
        else:
            print("UNSAT")

In [13]:
# Test code
c = Coloring()
c.readFile("test4.txt")
c.solve()

  2 3     0         
        3   2     6 
    5   5 3   5 7 4 
  4   5   5   6   3 
    4   5   6     3 
      2   5         
4   1       1 1     
4   1       1   4   
        6         4 
  4 4         4     
